# Multivariable Linear Regression

- 기존의 선형회귀는 x가 하나일 때 처리한 방식이다. 
- 그렇다면 여러 x개 있는 경우에는 어떻게 계산될까?

In [ ]:
import numpy as np
import torch

x_train = torch.FloatTensor([[73, 80, 75],
[93, 88, 93],
[89, 91, 90],
[96, 98, 100],
[73, 66, 70]])

y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

## Hypothesis

- 기존에는 `H(x) = wx + b`였다
- 이번에는 `H(x) = w1x1 + w2x2 + w3x3 + b`로 계산을 할 것 이다. 
- 문제는, 가중치를 위처럼 정적으로 계산하게 한다면 x가 길이 n인 벡터일 때, w도 n개를 수동으로 초기화시켜야 한다. 
- for문을 써도 매한가지다. 
- 이럴 때 쓰려고 `matmul`을 배웠다

In [ ]:
# hypothesis = x_train.matmul(W) + b